In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

## 1. How many Android ID’s have both customer and captain account

In [11]:
start_date = '20221201'
end_date = '20221231'

In [16]:
query_1 = f"""

select 
    *
from orders.order_logs_snapshot
where 
    yyyymmdd BETWEEN '{start_date}' and '{end_date}' 
    --and city_name = 'Bangalore'
    and order_status = 'dropped'
    and service_obj_service_name in ('Auto', 'Link')
""" 

In [ ]:
df_query_1 = pd.read_sql(query_1, conn)
df_query_1.head(5)

## 2. How many customer accounts are linked with an android ID

## 5. Captain starts with Android id “A” but ends with “B”

In [23]:
start_date = '20230116'
end_date = '20230122'

In [24]:
started_device = f"""
select
    yyyymmdd, service_obj_service_name service, 
    order_id, captain_id, customer_id, 
    captain_obj_device_id device_a, --customer_obj_device_device_id, 
    updated_epoch, 
    -- order_status, event_type,
    row_number() over(partition by order_id order by updated_epoch desc) seq_no
from orders.order_logs_immutable
where 
    yyyymmdd BETWEEN  '{start_date}' --]] DATE_FORMAT((DATE_TRUNC('week',CURRENT_DATE) - INTERVAL '7' day), '%Y%m%d')
    and '{end_date}' --]] DATE_FORMAT((DATE_TRUNC('week',CURRENT_DATE) - INTERVAL '1' day), '%Y%m%d')
    and service_obj_service_name in ('Auto', 'Link')
    and order_status = 'started' 
    and event_type = 'started'
    and (spd_fraud_flag != true OR spd_fraud_flag IS NULL)
"""

In [25]:
dropped_device = f"""
select
    yyyymmdd, service_obj_service_name service, 
    order_id, captain_id, customer_id, 
    captain_obj_device_id device_b, --customer_obj_device_device_id, 
    updated_epoch, 
    -- order_status, event_type,
    row_number() over(partition by order_id order by updated_epoch desc) seq_no
from orders.order_logs_immutable
where 
    yyyymmdd BETWEEN  '{start_date}' --]] DATE_FORMAT((DATE_TRUNC('week',CURRENT_DATE) - INTERVAL '7' day), '%Y%m%d')
    and '{end_date}' --]] DATE_FORMAT((DATE_TRUNC('week',CURRENT_DATE) - INTERVAL '1' day), '%Y%m%d')
    and service_obj_service_name in ('Auto', 'Link')
    and order_status = 'dropped' 
    and event_type = 'dropped'
    and (spd_fraud_flag != true OR spd_fraud_flag IS NULL)
"""

In [26]:
df_started_device = pd.read_sql(started_device, conn)
df_started_device.head(5)

,yyyymmdd,service,order_id,captain_id,customer_id,device_a,updated_epoch,seq_no
0,20230116,Auto,63c447093356566bfa43d4f8,60d9882b09aa9964313ab703,6108e99ab8bce1516112555a,6cbbc7c6b1d95105,1673808084005,1
1,20230116,Link,63c4477b05e9ee185a14b3f0,62bfc4ded33ad99a6e60103e,62a745617d79671cf1e7a19c,9f60b0c4a545f80d,1673808085690,1
2,20230116,Auto,63c448c43356566bfa43d6d9,5f476453b2c6ff4b5344a7f1,5fd9e8dd49fb269f96e7c01c,7d06fdcd0997f753,1673808334773,1
3,20230116,Link,63c4494359befa7065968b01,5c8a86ad5e042733c9ca9b53,5e555f79edd054bbe65b8429,863592053448751,1673808570441,1
4,20230116,Auto,63c449e92faf48746450958d,62e75f76cd87ee7e7b6e4acf,631c3f449b466b75e4acbb0c,866085037043258,1673808948255,1


In [28]:
df_started_device[df_started_device['seq_no']>1]

,yyyymmdd,service,order_id,captain_id,customer_id,device_a,updated_epoch,seq_no


In [27]:
df_dropped_device = pd.read_sql(dropped_device, conn)
df_dropped_device.head(5)

,yyyymmdd,service,order_id,captain_id,customer_id,device_b,updated_epoch,seq_no
0,20230116,Auto,63c4472e05e9ee185a14b392,5fc52e2f014147ed810ce7da,63807bf94b036a80ef0e6b85,afcef59d814cdd60,1673809562825,1
1,20230116,Link,63c447722faf487464509323,5e7088422f5b26aa3de1e12f,5c91bbb78c352421eaf30e93,2d701ab1e8a167fe,1673808972936,1
2,20230116,Link,63c4490159befa7065968aca,63bc02c0978ae123347e4555,63c448c120e3bb7fdec66077,fb8dd19f40dd7dee,1673809668002,1
3,20230116,Auto,63c44adc3356566bfa43d901,6299d4621b57e4d3efe5a518,62a987cf9d10c9c3838959ef,db221dcc49da75f6,1673812110619,1
4,20230116,Link,63c44af005e9ee185a14b79f,63b555386b2244244b93756d,5db95e59c54ee121252cf71d,00509872888a9ef4,1673810265236,1


In [29]:
df_dropped_device[df_dropped_device['seq_no']>1]

,yyyymmdd,service,order_id,captain_id,customer_id,device_b,updated_epoch,seq_no


In [35]:
a_device_b = df_dropped_device.merge(df_started_device, how = 'left', on = 'order_id' )

In [36]:
a_device_b

,yyyymmdd_x,service_x,order_id,captain_id_x,customer_id_x,device_b,updated_epoch_x,seq_no_x,yyyymmdd_y,service_y,captain_id_y,customer_id_y,device_a,updated_epoch_y,seq_no_y
0,20230116,Auto,63c4472e05e9ee185a14b392,5fc52e2f014147ed810ce7da,63807bf94b036a80ef0e6b85,afcef59d814cdd60,1673809562825,1,20230116,Auto,5fc52e2f014147ed810ce7da,63807bf94b036a80ef0e6b85,afcef59d814cdd60,1673807909475,1
1,20230116,Link,63c447722faf487464509323,5e7088422f5b26aa3de1e12f,5c91bbb78c352421eaf30e93,2d701ab1e8a167fe,1673808972936,1,20230116,Link,5e7088422f5b26aa3de1e12f,5c91bbb78c352421eaf30e93,2d701ab1e8a167fe,1673808206121,1
2,20230116,Link,63c4490159befa7065968aca,63bc02c0978ae123347e4555,63c448c120e3bb7fdec66077,fb8dd19f40dd7dee,1673809668002,1,20230116,Link,63bc02c0978ae123347e4555,63c448c120e3bb7fdec66077,fb8dd19f40dd7dee,1673808836926,1
3,20230116,Auto,63c44adc3356566bfa43d901,6299d4621b57e4d3efe5a518,62a987cf9d10c9c3838959ef,db221dcc49da75f6,1673812110619,1,20230116,Auto,6299d4621b57e4d3efe5a518,62a987cf9d10c9c3838959ef,db221dcc49da75f6,1673810319384,1
4,20230116,Link,63c44af005e9ee185a14b79f,63b555386b2244244b93756d,5db95e59c54ee121252cf71d,00509872888a9ef4,1673810265236,1,20230116,Link,63b555386b2244244b93756d,5db95e59c54ee121252cf71d,00509872888a9ef4,1673808836205,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3011282,20230120,Link,63ca0e63565e337603b359f5,5e12aa1689355c2289ab0f30,607680a879860efbf0b41649,308efb518971c208,1674187802343,1,20230120,Link,5e12aa1689355c2289ab0f30,607680a879860efbf0b41649,308efb518971c208,1674186670150,1
3011283,20230120,Link,63ca0e690ebcce3ed0c64913,5d99e98da1f0a71c605bbc4d,6050ea405e55c3d633ceeef9,3f4f48f31576d6bb,1674188225524,1,20230120,Link,5d99e98da1f0a71c605bbc4d,6050ea405e55c3d633ceeef9,3f4f48f31576d6bb,1674187136081,1
3011284,20230120,Link,63ca0e7b9ede1a236cc64dec,5d94e08e61bf5a14d090e454,631d5989a1189c39d8138d46,76f4eb718f18fa7e,1674187355361,1,20230120,Link,5d94e08e61bf5a14d090e454,631d5989a1189c39d8138d46,76f4eb718f18fa7e,1674186761553,1
3011285,20230120,Link,63ca0e8466ed1168b34005c3,63061b81e3fa3c311851911d,5d36e032c100a4655383c5d0,75b92f86a891f235,1674187186963,1,20230120,Link,63061b81e3fa3c311851911d,5d36e032c100a4655383c5d0,75b92f86a891f235,1674186836838,1


In [37]:
a_device_b['flag'] = np.where(a_device_b['device_b'] != a_device_b['device_a'] , 'Yes', 'No')

In [38]:
a_device_b[a_device_b['flag'].isin(['Yes'])]

,yyyymmdd_x,service_x,order_id,captain_id_x,customer_id_x,device_b,updated_epoch_x,seq_no_x,yyyymmdd_y,service_y,captain_id_y,customer_id_y,device_a,updated_epoch_y,seq_no_y,flag
